In [ ]:
import pandas as pd

In [ ]:
file_loc = input("Enter the file path: ")

# use accepted_clean.parquet

checklist = pd.read_parquet(file_loc)

In [ ]:
checklist.columns

In [ ]:
common_names = pd.read_csv(
    "/media/muskrat/T7 Shield/eco_data/v4/checklists/gbif/VernacularName.tsv",
    sep="\t",
    on_bad_lines="warn",
)

In [ ]:
common_names.columns

In [ ]:
common_names = common_names[["taxonID", "vernacularName", "language"]]

checklist = checklist[["taxonID", "acceptedName"]]

In [ ]:
# list unique values of language in common_names

common_names["language"].unique()

In [ ]:
# drop rows in common_names where language is not "en"

common_names = common_names[common_names["language"] == "en"]

In [ ]:
# merge checklist and common_names on taxonID

merged = pd.merge(checklist, common_names, on="taxonID", how="inner")

merged

In [ ]:
# drop rows in merged where acceptedName is not two words

merged = merged[merged["acceptedName"].str.count(" ") == 1]

merged

In [ ]:
# drop taxonID and language column from merged

merged = merged.drop(["taxonID", "language"], axis=1)

merged

In [ ]:
# rename acceptedName to scientific_name and vernacularName to common_name

merged = merged.rename(
    columns={"acceptedName": "scientific_name", "vernacularName": "common_name"}
)

merged

In [ ]:
species = pd.read_parquet(
    "/media/muskrat/T7 Shield/eco_data/v4/occurences/step_4/processing/step_3/filtered/filtered.parquet"
)

species

In [ ]:
# find rows in species where common_name is None

species[species["common_name"].isnull()]

In [ ]:
species_common = species[["scientific_name", "common_name"]]

species_common

In [ ]:
# find rows where common_name is None

common_null = species_common[species_common["common_name"].isnull()]

common_null

In [ ]:
merge_common = pd.merge(
    common_null, merged, on=["scientific_name", "common_name"], how="left"
)

merge_common

In [ ]:
# drop duplicates of scientific_name

merge_common = merge_common.drop_duplicates(subset="scientific_name", keep="first")

merge_common

In [ ]:
# find rows where common_name is None for merge_common

merge_common[merge_common["common_name"].isnull()]